## Project Cleanup and Maintenance

One of the best ways to utilize the Python API Workforce module is for cleanup and maintenance tasks. We'll review some of those here, which are featured in the "scripts" portion of our Github repo.

In [ ]:
import arcgis
from arcgis.apps import workforce
from arcgis.gis import GIS
import datetime
import pendulum
import csv
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("c765482bd0b9479b9104368da54df90d")
project = workforce.Project(item)
project

### Export assignments out to CSV

If you need to keep an archive of assignment data outside of the ArcGIS platform, exporting your assignments to a csv is a great solution!

In [ ]:
assignments = project.assignments.search(where="1=1")
assignments_to_export = []
# Take the assignment data, format it correctly if necessary, and assign it to the dict
for assignment in assignments:
    assignment_to_export = {}
    assignment_to_export["X"] = assignment.geometry["x"]
    assignment_to_export["Y"] = assignment.geometry["y"]
    assignment_to_export["DispatcherId"] = assignment.dispatcher_id
    assignment_to_export["WorkOrderId"] = assignment.work_order_id
    assignment_to_export["Status"] = assignment.status
    assignment_to_export["Description"] = assignment.description
    assignment_to_export["Notes"] = assignment.notes
    assignment_to_export["Priority"] = assignment.priority
    assignment_to_export["AssignmentType"] = assignment.assignment_type.name
    assignment_to_export["WorkerId"] = assignment.worker_id
    assignment_to_export["GlobalID"] = assignment.global_id
    assignment_to_export["Location"] = assignment.location
    assignment_to_export["Creator"] = assignment.creator
    assignment_to_export["Editor"] = assignment.editor
    assignment_to_export["DeclinedComment"] = assignment.declined_comment
    assignment_to_export["OBJECTID"] = assignment.object_id
    assignment_to_export["AssignmentRead"] = assignment.assignment_read
    # Append each field to the assignments to be exported
    assignments_to_export.append(assignment_to_export)
print("writing to csv")
# Create the CSV
with open("./exported.csv", 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ["OBJECTID",
                  "X",
                  "Y",
                  "Description",
                  "Status",
                  "Notes",
                  "Priority",
                  "AssignmentType",
                  "WorkOrderId",
                  "DueDate",
                  "WorkerId",
                  "GlobalID",
                  "Location",
                  "DeclinedComment",
                  "AssignedDate",
                  "AssignmentRead",
                  "InProgressDate",
                  "CompletedDate",
                  "DeclinedDate",
                  "PausedDate",
                  "DispatcherId",
                  "CreationDate",
                  "Creator",
                  "EditDate",
                  "Editor"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(assignments_to_export)
print("Completed")

### Delete Assignments

If you've used a project for a while, you might need to delete old assignments that fit a certain criteria. Let's manually decline an assignment here and then

In [ ]:
project.assignments.search()[0].update(status="Declined",declined_date=datetime.datetime.now(), declined_comment="not doing it")
print("Assignments before Deletion: " + str(len(project.assignments.search())))
project.assignments_item.layers[0].delete_features(where="status=4")
print("Assignments after Deletion: " + str(len(project.assignments.search())))

### Reset Stale Workers

A common task that administrators will need to accomplish is to reset the status of workers to not working. This portion takes workers who have not been edited (changed their location, status, or contact info) at or after a certain date and resets their status to "not working" (we have a dedicated script in the repo for this)

In [ ]:
workers = project.workers.search()
for worker in workers:
    print(worker.name + " Status: " + worker.status)

local_cutoff_date = pendulum.from_format("03/01/2020 12:00:00", "MM/DD/YYYY hh:mm:ss", tz="US/Eastern", formatter='alternative')
utc_dt = local_cutoff_date.in_tz('UTC')
formatted_date = utc_dt.strftime("%Y-%m-%d %H:%M:%S")

print("Querying workers")
where = f"{project._worker_schema.edit_date} < TIMESTAMP '{formatted_date}'"
workers = project.workers.search(where=where)
for worker in workers:
    worker.status = "not_working"
    
    
print("Updating workers")
project.workers.batch_update(workers)
print("Completed!")
workers = project.workers.search()
for worker in workers:
    print(worker.name + " Status: " + worker.status)

# The possibilities are endless!